## Get a csv file for each component stock of the SP 500 

## Imports

In [ ]:
import pandas as pd
import numpy as np
import os
import sys
import datetime
from os.path import expanduser

___
### Add to sys.path

Within the barchart_api module, references like:

```from barchartapi import barchart_api```

will not work

In [ ]:
WORKSPACE_DIR = '../..' 
PROJECT_DIR = f'{WORKSPACE_DIR}/barchartapi' 
WORKING_DIR = f'{PROJECT_DIR}/barchartapi' 
if WORKING_DIR not in sys.path:
    sys.path.append(WORKING_DIR)
if PROJECT_DIR not in sys.path:
    sys.path.append(PROJECT_DIR)
if WORKSPACE_DIR not in sys.path:
    sys.path.append(WORKSPACE_DIR)
import barchart_api as bcapi

___
### Create paths to where you will store the history data

In [ ]:
STOCKS_DIR = f'./temp_folder/sp'
try:
    os.makedirs(STOCKS_DIR)
except Exception as e:
    pass

In [ ]:
url_constituents = 'https://datahub.io/core/s-and-p-500-companies/r/constituents.csv'
df_constit = pd.read_csv(url_constituents)
df_constit.head()

___
### Create an instance of BcHist
1. select an enpoint prefix of free or paid
2. must set bar_type to minutes, hour or daily
3. must set interval
 * for minutes, set interval to 1, 5, 15, 30 or 60
 * for other types, see https://www.barchart.com/ondemand/api/getHistory

In [ ]:
# select and endpoint prefix
endpoint_prefix='free' # 'free' or 'paid'

# select a bartype and interval
# bar_type='minutes'
# interval=1
# bar_type='daily'
# interval=1
bar_type='minutes'
interval=30

# create an instance of BcHist
api_key = open(f'./temp_folder/{endpoint_prefix}_api_key.txt','r').read()
endpoint_type=f'{endpoint_prefix}_url'
bch = bcapi.BcHist(api_key, bar_type=bar_type, interval=interval,endpoint_type = endpoint_type)


___
### Main Loop to fetch history data from Barchart

#### Inputs
On each call to BcHist.get_history, provide:
 * a short name like:
  * a stock symbol like: SPY,USO,IBM, etc
  * a futures symbol like: CLJ18, GCG19, ESH17, etc
 * a beginning yyyymmdd integer
 * an ending yyyymmdd integer

#### Outputs
The get_history method will return a tuple, where:
 * tup[0] provides a status
 * tup[1] provides a pandas DataFrame of data, or None

In [ ]:
spydr_sector_names = ['XLE','XLU','XLK','XLB','XLP','XLY','XLI','XLC','XLV','XLF']
spydr_short_names = list(df_constit.Symbol)
short_names = spydr_short_names
beg_yyyymmdd = 20000101
end_yyyymmdd = 20190101

for short_name in short_names:
    sn_path = f'{STOCKS_DIR}/{short_name}.csv'
    if os.path.isfile(sn_path):
        print(f'BYPASSING: {short_name}')
        continue
    print(f'get_history: {short_name} BEGIN {datetime.datetime.now()}')
    tup = bch.get_history(short_name, beg_yyyymmdd, end_yyyymmdd)
    print(f'get_history: {short_name} WRITING DATA {datetime.datetime.now()}')
    tup[1].to_csv(sn_path,index=False)
    print(f'get_history: {short_name} END {datetime.datetime.now()}')
    

## END

In [ ]:
df = pd.read_csv(f'{os.getcwd()}/../../stock-rnn/df_final__SP500.csv')